In [44]:
import pandas as pd
from keras.utils import to_categorical

# loading data
train = pd.read_csv('data/digit-recognizer/train.csv')
test = pd.read_csv('data/digit-recognizer/test.csv')

In [45]:
# Split features and labels in training set
X = train.drop(labels=['label'], axis=1)
y = train['label']
# One hot encode labels
Y = to_categorical(y, num_classes=10)
# Normalize pixel data
X = X / 255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X = X.values.reshape(-1, 28, 28, 1)

# Definition des Modells

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
#from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.legacy import RMSprop  # legacy for m1,m2 macs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [47]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

# Trainieren des Modells

In [48]:
# Compiling model
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
# Fitting model
history = model.fit(X, Y, batch_size=100, epochs=1)

420/420 [==============================] - 7s 17ms/step - loss: 0.1703 - accuracy: 0.9487


# Evaluieren des Modells

In [49]:
# Split the data in training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

Y_pred = model.predict(X_test)

# Y_test and Y_pred need to be one-hot encoded
accuracy = accuracy_score(Y_test.argmax(axis=1), Y_pred.argmax(axis=1))

print("Accuracy on the test set:", accuracy)


329/329 [==============================] - 1s 3ms/step
Accuracy on the test set: 0.9823809523809524


# Erzeugen einer Submission Datei

In [50]:
import numpy as np

# Extract values from the DataFrame and reshape
test_values = test.values.reshape(-1, 28, 28, 1)

# Predict results
results = model.predict(test_values)

# One hot decoding: Select the index with the maximum probability
results = np.argmax(results, axis=1)

# Add columns ImageID and Label
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1, len(results) + 1), name="ImageId"), results], axis=1)

# Generate CSV file
submission.to_csv('data/digit-recognizer/submission.csv', index=False)

875/875 [==============================] - 2s 2ms/step
